In [24]:
import os
import sys
import time
import pennylane as qml

module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)
from measurement_memory import *
from pauli_grouping import *
from Experiment_Molecular import measurement_rotation, single_qubit_basis_rotation,\
                                 givens_rotation, phase_rotation
from Hamiltonians.utils import read_molecule_Hamiltonian
from pennylane import numpy as np

In [2]:
def To_pennylane(Hg:list):
    obs = []
    coeffs = []
    for i in range(len(Hg)):
        for j,p in enumerate(str_to_Pauli(Hg[i][0])):
            obs.append(p)
            coeffs.append(Hg[i][1][j])
    qmlH = qml.Hamiltonian(coeffs, obs)
    return qmlH

In [53]:
# Define ansatz
N = 12
p = 7
def ansatz(x):
    for i in range(N):
        qml.RX(x[i], wires=i)
    for j in range(p):
        for i in range(N-1):
            qml.CNOT(wires=[i,i+1])
        for i in range(N):
            qml.RX(x[i+N], wires=i)

In [54]:
H, G, P = read_molecule_Hamiltonian(mol="LiH", grouping_type="FG", directory='.')
dev = qml.device("lightning.qubit", wires=N)
@qml.qnode(dev, diff_method="parameter-shift")
def circuit_fg(x, p, g, qmlH):
    ansatz(x)
    measurement_rotation(grouping_type="FG", G=g, P=p)
    return qml.expval(qmlH)

def cost(x):
    E = 0
    for i,h in enumerate(H):
        qmlH = To_pennylane([h])
        g = G[i]
        p = P[i]
        E += circuit_fg(x, p, g, qmlH)
    return E

In [55]:
from pennylane import numpy as np

## First guess (stepsize=1.0)

In [59]:
opt = qml.AdagradOptimizer(stepsize=2.5)
max_steps=1000
init_params = 2*np.pi*np.random.rand(N*(p+1))

params = init_params
params.requires_grad = True
old_cost = 9999.999999
for i in range(max_steps):
    params = opt.step(cost, params)
    obj_value = cost(params)
    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, obj_value))
        if np.round(old_cost, 7) == np.round(obj_value, 7):
            break
        else:
            old_cost = obj_value
    
print("Optimized parameters: {}".format(params))
print("Optimized objective function value: {}".format(obj_value))

Cost after step     5: -5.2827727
Cost after step    10: -4.8792141
Cost after step    15: -4.7435256
Cost after step    20: -5.2639002
Cost after step    25: -5.3692735
Cost after step    30: -5.4668560
Cost after step    35: -5.4451696
Cost after step    40: -5.7258230
Cost after step    45: -5.5613574
Cost after step    50: -5.7467777
Cost after step    55: -5.9354635
Cost after step    60: -5.9308044
Cost after step    65: -5.8162709
Cost after step    70: -5.8844031
Cost after step    75: -6.0385359
Cost after step    80: -6.4273389
Cost after step    85: -6.3291866
Cost after step    90: -6.3113341
Cost after step    95: -6.2484224
Cost after step   100: -6.2540798
Cost after step   105: -6.1880466
Cost after step   110: -6.4108283
Cost after step   115: -6.3394680
Cost after step   120: -6.3608632
Cost after step   125: -6.3226377
Cost after step   130: -6.3676162
Cost after step   135: -6.3445621
Cost after step   140: -6.4575904
Cost after step   145: -6.4311207
Cost after ste

## small step calculation

In [42]:
opt = qml.AdagradOptimizer(stepsize=0.5)
max_steps=1000

old_cost = 9999.999999
for i in range(max_steps):
    params = opt.step(cost, params)
    obj_value = cost(params)
    print("Cost after step {:5d}: {: .7f}".format(i + 1, obj_value))
    if np.round(old_cost, 7) == np.round(obj_value, 7):
        break
    else:
        old_cost = obj_value
    
print("Optimized parameters: {}".format(params))
print("Optimized objective function value: {}".format(obj_value))

Cost after step     1: -5.5583138
Cost after step     2: -6.0417008
Cost after step     3: -6.0981723
Cost after step     4: -6.1494732
Cost after step     5: -6.1937425
Cost after step     6: -6.2914793
Cost after step     7: -6.3784622
Cost after step     8: -6.4216777
Cost after step     9: -6.4610057
Cost after step    10: -6.4883271
Cost after step    11: -6.5061884
Cost after step    12: -6.5162024
Cost after step    13: -6.5252601
Cost after step    14: -6.5246806
Cost after step    15: -6.5206180
Cost after step    16: -6.5252767
Cost after step    17: -6.5386298
Cost after step    18: -6.5448955
Cost after step    19: -6.5490305
Cost after step    20: -6.5515540
Cost after step    21: -6.5534579
Cost after step    22: -6.5548879
Cost after step    23: -6.5560689
Cost after step    24: -6.5570589
Cost after step    25: -6.5579367
Cost after step    26: -6.5587370
Cost after step    27: -6.5594998
Cost after step    28: -6.5602509
Cost after step    29: -6.5610182
Cost after ste